# GridSearchCVによるパラメタチューニング
Boston house-pricesデータセットを題材にElasticNetのパラメタをGridSearchCVでチューニング

In [ ]:
from sklearn.datasets import load_boston
from sklearn.model_selection import train_test_split
from sklearn.linear_model import ElasticNet
from sklearn.model_selection import GridSearchCV
from sklearn.metrics import r2_score

import numpy as np

%matplotlib inline
import matplotlib.pyplot as plt

In [ ]:
# Boston house-pricesのロード
boston = load_boston()

X = boston.data
y = boston.target

In [ ]:
# 学習データと評価データに分割（分割比率8:2）
(X_train, X_test, y_train, y_test) = train_test_split(X, y, test_size=0.2, random_state=0)

In [ ]:
# デフォルトパラメタ
elastic_net = ElasticNet()
elastic_net.fit(X_train, y_train)

# 評価データでのR2スコアを算出
pred = elastic_net.predict(X_test)
score = r2_score(y_test, pred)

print('ElasticNet(default) R2 score: %.3f' % score)

In [ ]:
# チューニングモデル
estimator = ElasticNet()

# モデルのパラメタ候補を定義
alpha_params = np.logspace(-3, 3, 7)
l1_ratio_params = np.arange(0.1, 1.0, 0.1)

paramters = {
    'alpha': alpha_params,
    'l1_ratio': l1_ratio_params
}

# パラメタチューニング
model_tuning = GridSearchCV(
    estimator = estimator,     # 識別器
    param_grid = paramters,    # パラメタ
    refit = True,              # bestモデルを学習データ全体で再学習
    cv = 5,                    # Cross validationの分割数
    n_jobs = -1,               # 並列実行数(-1: コア数で並列実行)
    verbose = 0                # メッセージ出力レベル
)

model_tuning.fit(X_train, y_train)

In [ ]:
def PlotGridSearchScores(model_tuning, grid_param_1, grid_param_2, name_param_1, name_param_2):
    scores_mean = model_tuning.cv_results_['mean_test_score']
    scores_mean = np.array(scores_mean).reshape(len(grid_param_2),len(grid_param_1))

    _, ax = plt.subplots(1,1)

    # param1をx軸、param2を折れ線グラフで表現
    for idx, val in enumerate(grid_param_2):
        ax.plot(grid_param_1, scores_mean[idx,:], '-o', label= name_param_2 + ': ' + str(val))

    ax.set_title("Grid Search R2 Scores", fontsize=20, fontweight='bold')
    ax.set_xlabel(name_param_1, fontsize=16)
    ax.set_ylabel('CV Average Score', fontsize=16)
    ax.legend(loc="upper right", bbox_to_anchor=(1.55, 0.95, 0.5, .100), fontsize=15)
    ax.grid('on')

In [ ]:
# チューニング結果を描画
PlotGridSearchScores(model_tuning, l1_ratio_params, alpha_params, 'l1_ratio', 'alpha')

In [ ]:
# チューニングしたモデルを評価
elastic_net_tuned = model_tuning.best_estimator_

# 評価データでのR2スコアを算出
pred = elastic_net_tuned.predict(X_test)
score = r2_score(y_test, pred)

print('ElasticNet(tuned) R2 score: %.3f' % score)